<a href="https://colab.research.google.com/github/liampearson/Google_Mobility_Data/blob/main/mobility_data_by_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
from os.path import join
import math

#from google.colab import drive
#drive.mount('/content/drive/')
#write_directory = '/content/drive/My Drive/covid19data/'

# authentication for google sheets
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())



In [20]:
# region csv is located at: https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip

#download the zip file
!wget https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip

#unzip the file
!unzip Region_Mobility_Report_CSVs.zip

#get the AU file
REGION = "AU"
df2020 = pd.read_csv("2020_"+REGION+"_Region_Mobility_Report.csv")
df2021 = pd.read_csv("2021_"+REGION+"_Region_Mobility_Report.csv")
df = df2020.append(df2021)

df.head()

--2021-08-11 21:56:50--  https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip
Resolving www.gstatic.com (www.gstatic.com)... 74.125.204.94, 2404:6800:4008:c07::5e
Connecting to www.gstatic.com (www.gstatic.com)|74.125.204.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54500203 (52M) [application/zip]
Saving to: ‘Region_Mobility_Report_CSVs.zip.4’

Region_Mobility_Rep 100%[===================>]  51.97M  19.2MB/s    in 2.7s    

2021-08-11 21:56:53 (19.2 MB/s) - ‘Region_Mobility_Report_CSVs.zip.4’ saved [54500203/54500203]

Archive:  Region_Mobility_Report_CSVs.zip
replace 2020_AE_Region_Mobility_Report.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: 2020_AE_Region_Mobility_Report.csv  
  inflating: 2020_AF_Region_Mobility_Report.csv  
  inflating: 2020_AG_Region_Mobility_Report.csv  
  inflating: 2020_AO_Region_Mobility_Report.csv  
  inflating: 2020_AR_Region_Mobility_Report.csv  
  inflating: 2020_AT_Region_Mobility_Report.cs

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-15,4.0,3.0,-2.0,3.0,3.0,0.0
1,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-16,3.0,5.0,9.0,3.0,-1.0,0.0
2,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-17,-1.0,0.0,-6.0,7.0,17.0,-2.0
3,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-18,-3.0,-2.0,-13.0,7.0,14.0,-1.0
4,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-19,-1.0,-1.0,-6.0,8.0,13.0,-1.0


In [21]:
#notes for AU:
#df.sub_region_1 = State = [, 'Australian Capital Territory', 'New South Wales', 'Northern Territory', 'Queensland', 'South Australia', 'Tasmania', 'Victoria', 'Western Australia']
#df.sub_region_2 = councils
#df.metro_area = null
#df.iso_3166_2_code.unique() = [nan, 'AU-ACT', 'AU-NSW', 'AU-NT', 'AU-QLD', 'AU-SA', 'AU-TAS','AU-VIC', 'AU-WA']
#df[df.iso_3166_2_code=='AU-NSW'].date.tail()
#df[df.date==max(df.date)]

notes for AU:

df.sub_region_1 = State = [, 'Australian Capital Territory', 'New South Wales', 'Northern Territory', 'Queensland', 'South Australia', 'Tasmania', 'Victoria', 'Western Australia']

df.sub_region_2 = councils

df.metro_area = null

df.iso_3166_2_code.unique() = [nan, 'AU-ACT', 'AU-NSW', 'AU-NT', 'AU-QLD', 'AU-SA', 'AU-TAS','AU-VIC', 'AU-WA']

df[df.iso_3166_2_code=='AU-NSW'].date.tail()

df[df.date==max(df.date)]


## Get place_id lookup (equivalent of index_v)
Google assigned a unique `place_id` for each Country / sub_region_1 / sub_region_2...etc

For each unique place_id, we will get 3 fields:
- Country / sub_region_1 / sub_region_2

In [22]:
# initialise an empty dataframe for the place_id lookup
place_id_lookup = pd.DataFrame()

for placeid in df.place_id.unique():
  #ensure there is only one country_region for this placeid
  if (len(df[df['place_id']==placeid]['country_region'].unique())!=1):
    print("ERROR!!!")
  country_region = df[df['place_id']==placeid]['country_region'].unique()[0]

  #ensure there is only one iso_3166_2_code for this placeid
  if (len(df[df['place_id']==placeid]['iso_3166_2_code'].unique())!=1):
    print("ERROR!!!")
  iso_3166_2_code = df[df['place_id']==placeid]['iso_3166_2_code'].unique()[0]

  #ensure there is only one sub_region_1 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_1'].unique())!=1):
    print("ERROR!!!")
  sub_region_1 = df[df['place_id']==placeid]['sub_region_1'].unique()[0]

  #ensure there is only one sub_region_2 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_2'].unique())!=1):
    print("ERROR!!!")
  sub_region_2 = df[df['place_id']==placeid]['sub_region_2'].unique()[0]


  new_row = {'place_id'       : placeid,
             'iso_3166_2_code': iso_3166_2_code,
             'country_region' : country_region, 
             'sub_region_1'   : sub_region_1, 
             'sub_region_2'   : sub_region_2
            }

  #append row to the dataframe
  place_id_lookup = place_id_lookup.append(new_row, ignore_index=True)

#inspect the place_id_lookup
place_id_lookup

,country_region,iso_3166_2_code,place_id,sub_region_1,sub_region_2
0,Australia,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,NaN,NaN
1,Australia,AU-ACT,ChIJSxCboN9MFmsRA3huXDhEWOc,Australian Capital Territory,NaN
2,Australia,AU-NSW,ChIJDUte93TLDWsRLZ_EIhGvgBc,New South Wales,NaN
3,Australia,NaN,ChIJPXdpvMSnn2sRamkBGce1LIM,New South Wales,Armidale Dumaresq Council
4,Australia,NaN,ChIJGSMhcwKCkGsRWLKq2JtdcD0,New South Wales,Ballina Shire Council
...,...,...,...,...,...
296,Australia,NaN,ChIJywG3noSlMioRCz8TieOMH0Q,Western Australia,Town of Cambridge
297,Australia,NaN,ChIJzy2ctjekMioRCcmcWIaa8IQ,Western Australia,Town of Claremont
298,Australia,NaN,ChIJxf2tdx-kMioRJOZpQ6GJ2hQ,Western Australia,Town of Cottesloe
299,Australia,NaN,ChIJk9DO_Yoq9SsRX_KCepEBXlA,Western Australia,Town of Port Hedland


In [23]:
# initialise an empty dataframe for the place_id lookup
place_id_lookup = pd.DataFrame()
place_id_to_lga= {} #dictionary

for placeid in df.place_id.unique():
  #ensure there is only one country_region for this placeid
  if (len(df[df['place_id']==placeid]['country_region'].unique())!=1):
    print("ERROR!!!")
  country_region = df[df['place_id']==placeid]['country_region'].unique()[0]

  #ensure there is only one iso_3166_2_code for this placeid
  if (len(df[df['place_id']==placeid]['iso_3166_2_code'].unique())!=1):
    print("ERROR!!!")
  iso_3166_2_code = df[df['place_id']==placeid]['iso_3166_2_code'].unique()[0]

  #ensure there is only one sub_region_1 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_1'].unique())!=1):
    print("ERROR!!!")
  sub_region_1 = df[df['place_id']==placeid]['sub_region_1'].unique()[0]

  #ensure there is only one sub_region_2 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_2'].unique())!=1):
    print("ERROR!!!")
  sub_region_2 = df[df['place_id']==placeid]['sub_region_2'].unique()[0]


  new_row = {'place_id'       : placeid,
             'iso_3166_2_code': iso_3166_2_code,
             'country_region' : country_region, 
             'sub_region_1'   : sub_region_1, 
             'sub_region_2'   : sub_region_2
            }

  #append row to the dataframe
  place_id_lookup = place_id_lookup.append(new_row, ignore_index=True)

  place_id_to_lga[sub_region_2] = placeid

#inspect the place_id_lookup
place_id_lookup

,country_region,iso_3166_2_code,place_id,sub_region_1,sub_region_2
0,Australia,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,NaN,NaN
1,Australia,AU-ACT,ChIJSxCboN9MFmsRA3huXDhEWOc,Australian Capital Territory,NaN
2,Australia,AU-NSW,ChIJDUte93TLDWsRLZ_EIhGvgBc,New South Wales,NaN
3,Australia,NaN,ChIJPXdpvMSnn2sRamkBGce1LIM,New South Wales,Armidale Dumaresq Council
4,Australia,NaN,ChIJGSMhcwKCkGsRWLKq2JtdcD0,New South Wales,Ballina Shire Council
...,...,...,...,...,...
296,Australia,NaN,ChIJywG3noSlMioRCz8TieOMH0Q,Western Australia,Town of Cambridge
297,Australia,NaN,ChIJzy2ctjekMioRCcmcWIaa8IQ,Western Australia,Town of Claremont
298,Australia,NaN,ChIJxf2tdx-kMioRJOZpQ6GJ2hQ,Western Australia,Town of Cottesloe
299,Australia,NaN,ChIJk9DO_Yoq9SsRX_KCepEBXlA,Western Australia,Town of Port Hedland


In [24]:
def get_timestamp():
  from datetime import datetime, timezone

  #get sydney timezone
  import pytz
  sydney_tz = pytz.timezone('Australia/Sydney')
  now = datetime.now(sydney_tz)

  # format to dd/mm/YY H:M:S with timezone
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S %z")
  return dt_string

get_timestamp()

'12/08/2021 07:58:00 +1000'

In [25]:
def write_to_gsheet(workbook_title, worksheet_title, data):
  # open spreadsheet
  sh = gc.open(workbook_title)

  #select an existing worksheet (tab)
  worksheet = sh.worksheet(worksheet_title)

  #delete that worksheet
  sh.del_worksheet(worksheet)

  #recreate worksheet (tab) and populate with dataframe
  worksheet = sh.add_worksheet(title=worksheet_title, rows="1000", cols="200")
  set_with_dataframe(worksheet, data, include_index=True)

In [26]:
# write the place id's to sheet
write_to_gsheet('place_id_lookup', 'place_ids', place_id_lookup)

APIError: ignored

# Process specific to Australia Data
1. Pivot the data
2. Move the 'Australia' label down to the other levels
3. Move the State labels down to the sub_region_2

In [27]:
df.sub_region_1.copy().dropna().unique()

array(['Australian Capital Territory', 'New South Wales',
       'Northern Territory', 'Queensland', 'South Australia', 'Tasmania',
       'Victoria', 'Western Australia'], dtype=object)

In [28]:
CATEGORIES = ['retail_and_recreation_percent_change_from_baseline',
              'grocery_and_pharmacy_percent_change_from_baseline',
              'parks_percent_change_from_baseline',
              'transit_stations_percent_change_from_baseline',
              'workplaces_percent_change_from_baseline',
              'residential_percent_change_from_baseline'
            ]

# update the fields for AU (sub regions are null for Country)
# want unique state values excluding null
#df.loc[(df.sub_region_1.isnull()) & (df.sub_region_2.isnull()) & (df.iso_3166_2_code.isnull()) & (df['country_region']=='Australia'), 'sub_region_1'] = 'AUS'
#df.loc[(df.sub_region_2.isnull()) & (df.iso_3166_2_code.isnull()) & (df['country_region']=='Australia'), 'sub_region_2'] = 'AUS'

# move the iso_3166_2_code (i.e. state) into sub_region_2
# want unique state values excluding null
for state in df.iso_3166_2_code.copy().dropna().unique():
  df.loc[df['iso_3166_2_code']==state, 'sub_region_2'] = state[3:]


cols = ['sub_region_2']

# for each state
for state in df.sub_region_1.copy().dropna().unique():
  print("processing "+state+"...")

  # for each category of mobility data, pivot on that category
  for category in CATEGORIES:
    print("processing {}".format(category))
    mobility_data = df[df['sub_region_1']==state].pivot(index = 'date', columns = cols, values=category)
    
    #remove spaces from state name
    state = state.replace(" ","_").lower()

    #get rolling average
    rolling_mean_df = mobility_data.rolling(7).mean()

    #add timestamp
    mobility_data['last_updated'] = get_timestamp()
  # mobility_data_pivoted['last_updated'] = get_timestamp()
    rolling_mean_df['last_updated'] = get_timestamp()

    #write to googlesheets
    write_to_gsheet(workbook_title=state+'_mobility_data',
                  worksheet_title = category.split('_')[0],
                  data = mobility_data)
    
    write_to_gsheet(workbook_title=state+'_mobility_data_rolling_7day_average',
                  worksheet_title = category.split('_')[0],
                  data = rolling_mean_df)

processing Australian Capital Territory...
processing retail_and_recreation_percent_change_from_baseline
processing grocery_and_pharmacy_percent_change_from_baseline
processing parks_percent_change_from_baseline
processing transit_stations_percent_change_from_baseline
processing workplaces_percent_change_from_baseline
processing residential_percent_change_from_baseline
processing New South Wales...
processing retail_and_recreation_percent_change_from_baseline
processing grocery_and_pharmacy_percent_change_from_baseline
processing parks_percent_change_from_baseline
processing transit_stations_percent_change_from_baseline
processing workplaces_percent_change_from_baseline
processing residential_percent_change_from_baseline
processing Northern Territory...
processing retail_and_recreation_percent_change_from_baseline
processing grocery_and_pharmacy_percent_change_from_baseline
processing parks_percent_change_from_baseline
processing transit_stations_percent_change_from_baseline
processing

In [29]:
print("SUCCESS - script has run as of {}".format(get_timestamp()))

SUCCESS - script has run as of 12/08/2021 08:05:07 +1000
